## Problem Set 4

**Q1**

The solution solves: 

$$\mathbb{P}[|E_{out}-E_{in}|>\epsilon] \leq 4(2N)^{d_{vc}}\mathcal{e}^{-\frac{1}{8}\epsilon^2N} \leq 0.05$$

for $N$, where $\epsilon=0.05$, $d_{vc}=10$, and $N>d_{vc}$.

In [35]:
import math
from scipy.optimize import fsolve

def mH(N, dvc):
    return float(N)**dvc

def generalize(epsilon, dvc, err):
    def f(N):
        return (
            4 * mH(2*N, dvc) 
            * math.exp(-1 / 8 * epsilon**2 * float(N)) 
            - err
        )
    return f

f = generalize(0.05, 10, 0.05)

fsolve(f, 400000)

array([ 452956.8647231])

**Q2**

For a very large N, Devroye provides the tightest bound on the generalization error, $\epsilon$.

In [32]:
def bounds(N, dvc, delta):
    def og_vc(epsilon):
        return (
            math.sqrt(8 / N * math.log(4 * mH(2*N, dvc) / delta)) 
            - epsilon
        )
    def rademacher(epsilon):
        return (
            math.sqrt(2 * math.log(2 * N * mH(N, dvc)) / N) 
            + math.sqrt(2 / N * math.log(1 / delta) + 1 / N) 
            - epsilon
        )
    def parrondo(epsilon):
        return (
            math.sqrt(1 / N * 
                      (2 * epsilon 
                       + math.log(6 * mH(2*N, dvc) / delta))
                     ) 
            - epsilon
        )
    def devroye(epsilon):
        return (
            math.sqrt(
                1 / (2*N) * (
                    4 * epsilon * (1 + epsilon) 
                    + math.log(4/delta) + 2*dvc*math.log(N))
            ) 
            - epsilon
        )
    
    return {
        "og_vc": og_vc,
        "rademacher": rademacher,
        "parrondo": parrondo,
        "devroye": devroye
    }
        
b = bounds(10000.0, 50, 0.05)

print('Original VC bound (N = 10,000): {}'.format(
        fsolve(b['og_vc'], 0)))
print('Rademacher penalty bound (N = 10,000): {}'.format(
        fsolve(b['rademacher'], 0)))
print('Parrondo and Van den Broek bound (N = 10,000): {}'.format(
        fsolve(b['parrondo'], 0)))
print('Devroye bound (N = 10,000): {}'.format(
        fsolve(b['devroye'], 0)))

Original VC bound (N = 10,000): [ 0.63217492]
Rademacher penalty bound (N = 10,000): [ 0.3331727]
Parrondo and Van den Broek bound (N = 10,000): [ 0.22369829]
Devroye bound (N = 10,000): [ 0.21522805]


**Q3**

For small N, the least bound on the generalization error is given by Parrondo and Van den Broek.

In [34]:
b = bounds(5.0, 50, 0.05)

print('Original VC bound (N = 5): {}'.format(
        fsolve(b['og_vc'], 0)))
print('Rademacher penalty bound (N = 5): {}'.format(
        fsolve(b['rademacher'], 0)))
print('Parrondo and Van den Broek bound (N = 5): {}'.format(
        fsolve(b['parrondo'], 0)))
print('Devroye bound (N = 5): {}'.format(
        fsolve(b['devroye'], 0)))

Original VC bound (N = 5): [ 13.82816148]
Rademacher penalty bound (N = 5): [ 6.93660526]
Parrondo and Van den Broek bound (N = 5): [ 5.10136198]
Devroye bound (N = 5): [ 5.59312554]


**Q4**

In [2]:
from lfd import Data, OLS
import numpy as np
import random as rd

N_train = 2
N_simulations = 10000

weights = []
X_list, Y_list = [], []
gD_list = []

for _ in range(N_simulations):
    training_set = Data([N_train, 1], intercept=False)
    training_set.Y = np.array([np.sin(np.pi * row[0]) for row in training_set.X])

    linreg = OLS()
    # regress X on Y with intercept set to 0.
    linreg.run(training_set.X, training_set.Y)

    weights.append(linreg.weights[0])
    
    # for later:
    gD_list.extend((linreg.weights[0] * training_set.X))
    X_list.extend(training_set.X)
    Y_list.extend(training_set.Y)

mean_weight = np.mean(weights)

print('The expected value of the hypothesis is: {}x'.format(
        round(mean_weight, 2)))

The expected value of the hypothesis is: 1.43x


**Q5-6**

In [3]:
variance_list = []
bias_list = []

for i in range(len(X_list)):
    point_var = (gD_list[i] - mean_weight * X_list[i])**2
    point_bias = (Y_list[i] - mean_weight * X_list[i])**2
    variance_list.append(point_var)
    bias_list.append(point_bias)

print('The bias is: {}'.format(np.mean(bias_list)))
print('The variance is: {}'.format(np.mean(variance_list)))

The bias is: 0.2714601164590224
The variance is: 0.20438896192084355


**Q7**

In [4]:
a_weights = []
b_weights = []
c_weights = []
d_weights = []
e_weights = []


def mse(hypothesis, actual):
    return np.mean(np.square(np.subtract(hypothesis, actual)))


for _ in range(N_simulations):
    training_set = Data([N_train, 2], intercept=True)
    training_set.Y = np.array([np.sin(np.pi * row[1]) for row in training_set.X])

    # add an x^2 column
    training_set.add_columns([
            np.multiply(training_set.X[:, 1], training_set.X[:, 1])
        ])

    linreg = OLS()
    
    # h(x) = b
    linreg.run(training_set.X[:, [0]], training_set.Y)
    a_weights.append(linreg.weights)
    
    # h(x) = ax
    linreg.run(training_set.X[:, [1]], training_set.Y)
    b_weights.append(linreg.weights)
    
    # h(x) = ax + b
    linreg.run(training_set.X[:, [0, 1]], training_set.Y)
    c_weights.append(linreg.weights)
    
    # h(x) = ax^2
    linreg.run(training_set.X[:, [2]], training_set.Y)
    d_weights.append(linreg.weights)

    # h(x) = ax^2 + b
    linreg.run(training_set.X[:, [0, 2]], training_set.Y)
    e_weights.append(linreg.weights)

a_weights = np.mean(a_weights, axis=0)
b_weights = np.mean(b_weights, axis=0)
c_weights = np.mean(c_weights, axis=0)
d_weights = np.mean(d_weights, axis=0)
e_weights = np.mean(e_weights, axis=0)

N_test = 10000

test_set = Data([N_test, 2], intercept=True)
test_set.Y = np.array([np.sin(np.pi * row[1]) for row in test_set.X])

test_set.add_columns([
        np.multiply(test_set.X[:, 1], test_set.X[:, 1])
    ])

print(mse(np.dot(test_set.X[:, [0]], a_weights), test_set.Y))
print(mse(np.dot(test_set.X[:, [1]], b_weights), test_set.Y))
print(mse(np.dot(test_set.X[:, [0, 1]], c_weights), test_set.Y))
print(mse(np.dot(test_set.X[:, [2]], d_weights), test_set.Y))
print(mse(np.dot(test_set.X[:, [0, 2]], e_weights), test_set.Y))

0.507375568176
0.270689796785
0.208903640837
0.512868032904
4.9585857921


**Q8**

$$\mathcal{m_H}(N+1)=2\mathcal{m_H}(N)-\binom{N}{q} \\
=2(2\mathcal{m_H}(N-1)-\binom{N-1}{q})-\binom{N}{q} \\
=2^2\mathcal{m_H}(N-1)-(2\binom{N-1}{q}+\binom{N}{q}) \\
=2^{N}\mathcal{m_H}(N-(N-1))-(2^{N-1}\binom{N-(N-1)}{q}+...+2^1\binom{N-1}{q}+2^0\binom{N}{q}) \\
=2^{N}\mathcal{m_H}(1) - \sum_{n=0}^{N-1} 2^n \binom{N-n}{q} \\
=2^{N} \cdot 2 -\sum_{n=0}^{N-1} 2^n \binom{N-n}{q} \\
=2^{N+1} -\sum_{n=0}^{N-1} 2^n \binom{N-n}{q} \\
$$

Or in terms of $N$:

$$\mathcal{m_H}(N)= 2^{N} -\sum_{n=0}^{N-2} 2^n \binom{N-1-n}{q}$$

The VC dimension of a hypothesis set $\mathcal{H}$ is the largest (positive integer) value of $N$ for which $\mathcal{m_H}(N)=2^N$. In other words, we are looking for the largest $N$ such that:

$$\sum_{n=0}^{N-2} 2^n \binom{N-1-n}{q}=0$$

If $N>q$, this series will have at least one positive term. If $N=q$, all the terms equal zero. So the VC dimension, $d_{vc}(\mathcal{H})=q$.